In [17]:
import os
import re
import cv2
import pandas as pd
import numpy as np

In [18]:
current_directory = "./assets/pack_1/"
contents = os.listdir(current_directory)

df = pd.DataFrame([], columns=["image", "fractal_dimension"])

In [19]:
def sorting_function(el: str) -> tuple[int, int]:
    el = re.split("[._]", el)
    return int(el[1]), int(el[2])


def box_count(Z: np.ndarray, k: int):
    S = np.add.reduceat(
        np.add.reduceat(Z, np.arange(0, Z.shape[0], k), axis=0),
        np.arange(0, Z.shape[1], k),
        axis=1
    )
    return len(np.where((S > 0) & (S < k * k))[0])


def fractal_dimension(Z: np.ndarray, threshold: float = 0.9):
    Z = (Z < threshold)
    p = min(Z.shape)
    n = 2 ** np.floor(np.log(p) / np.log(2))
    n = int(np.log(n) / np.log(2))
    sizes = 2 ** np.arange(n, 1, -1)
    counts = []
    for size in sizes:
        counts.append(box_count(Z, size))
    coefficients = np.polyfit(np.log(sizes), np.log(counts), 1)
    return -coefficients[0]

In [20]:
for img in sorted(contents, key=sorting_function):
    # image = plt.imread(current_directory + img, 0)
    image = cv2.imread(current_directory + img, cv2.IMREAD_GRAYSCALE)

    image = cv2.GaussianBlur(image, (1, 1), 0)
    image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 17, 1) / 255

    image[0, 1] = 1
    image[0, 6:12] = 1
    fractal_d = fractal_dimension(image, threshold=0.25)
    df.loc[len(df.index)] = [img, fractal_d]

In [21]:
df.head()
df.tail()

,image,fractal_dimension
250,finger_51_1.png,1.828950
251,finger_51_2.png,1.828961
252,finger_51_3.png,1.828558
253,finger_51_4.png,1.828350
254,finger_51_5.png,1.827764


In [22]:
df.to_csv("assets/fractal.csv", index=False)